In [1]:
import pandas as pd
from pathlib import Path
import json
from pprint import pprint
import math
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm

DATA_DIR = Path("data/benchmarking_2")

In [2]:
def get_in_road_percentage(steps_df: pd.DataFrame) -> float:
    ret = steps_df["out_of_road"].value_counts(normalize=True).at[False]
    return ret

In [3]:
def get_n_sidewalk_crashes(steps_df: pd.DataFrame) -> int:
    """Count number of crash episodes to not count same crash multiple times"""
    try:
        # count number of times "crash" becomes True for some time
        n_crashes = steps_df["crash_sidewalk"].diff().value_counts().at[True]

        # need to divide by 2 beacouse diff counts twice
        n_crashes /= 2

        # just in case crash is last episode and we have 3.5 crash episodes make it 4
        n_crashes = math.ceil(n_crashes)
    except KeyError:
        n_crashes = 0

    return n_crashes

In [4]:
def process_steps(steps_infos: list) -> dict:
    """Accepts a list of steps and returns a dict of interesting data"""

    steps_df = pd.DataFrame(steps_infos)
    steps_data = {}
    last_step = steps_df.iloc[-1]

    steps_data = {
        "termination.arrive_dest": last_step["arrive_dest"],
        "termination.timeout": last_step["max_step"],
        "route_completion": last_step["route_completion"],
        "seed": last_step["env_seed"],
        "in_road_percentage": get_in_road_percentage(steps_df),
        "n_sidewalk_crashes": get_n_sidewalk_crashes(steps_df),
    }

    return steps_data

In [5]:
def get_scenarios_df(dir: Path):
    paths = list(dir.glob("*/*.json"))

    scenarios = []
    for file_path in tqdm(paths):

        with open(file_path, "r") as f:
            scenario_data = json.load(f)

        dir = file_path.__str__().split("/")[-2]
        _, dr, _, dt = dir.split("_")
        scenario_data["decision_repeat"] = int(dr)
        scenario_data["dt"] = float(dt)

        steps_infos = scenario_data.pop("steps_infos")
        scenario_data.update(process_steps(steps_infos))
        scenarios.append(scenario_data)
    return pd.DataFrame(scenarios)

In [6]:
df = get_scenarios_df(DATA_DIR)

100%|██████████| 1200/1200 [00:01<00:00, 765.99it/s]


In [7]:
df = df.set_index(
    [
        "dt",
        "decision_repeat",
        "seed",
    ],
    verify_integrity=True,
)

df = df.sort_index()

In [8]:
df

def.map_seq  \
dt   decision_repeat seed                                                      
0.02 5               0     [{'id': 'I', 'pre_block_socket_index': None}, ...   
                     1     [{'id': 'I', 'pre_block_socket_index': None}, ...   
                     2     [{'id': 'I', 'pre_block_socket_index': None}, ...   
                     3     [{'id': 'I', 'pre_block_socket_index': None}, ...   
                     4     [{'id': 'I', 'pre_block_socket_index': None}, ...   
...                                                                      ...   
0.04 20              95    [{'id': 'I', 'pre_block_socket_index': None}, ...   
                     96    [{'id': 'I', 'pre_block_socket_index': None}, ...   
                     97    [{'id': 'I', 'pre_block_socket_index': None}, ...   
                     98    [{'id': 'I', 'pre_block_socket_index': None}, ...   
                     99    [{'id': 'I', 'pre_block_socket_index': None}, ...   

                                                                 def.bv_data  \
dt   decision_repeat seed                                                      
0.02 5               0     [{'length': 4.515, 'width': 1.852, 'height': 1...   
                     1     [{'length': 4.515, 'width': 1.852, 'height': 1...   
                     2     [{'length': 4.515, 'width': 1.852, 'height': 1...   
                     3     [{'length': 4.515, 'width': 1.852, 'height': 1...   
                     4     [{'length': 4.515, 'width': 1.852, 'height': 1...   
...                                                                      ...   
0.04 20              95    [{'length': 4.515, 'width': 1.852, 'height': 1...   
                     96    [{'length': 4.515, 'width': 1.852, 'height': 1...   
                     97    [{'length': 4.515, 'width': 1.852, 'height': 1...   
                     98    [{'length': 4.515, 'width': 1.852, 'height': 1...   
                     99    [{'length': 4.515, 'width': 1.852, 'height': 1...   

                           def.max_steps      start_ts  initialized_ts  \
dt   decision_repeat seed                                                
0.02 5               0              3364  26007.062831    26007.531759   
                     1              1892  26014.706182    26015.015510   
                     2              3574  26020.585997    26020.946454   
                     3              2092  26026.570167    26026.851898   
                     4              2815  26031.646844    26031.908181   
...                                  ...           ...             ...   
0.04 20              95              423  26751.146416    26751.508414   
                     96              330  26760.923083    26761.216448   
                     97              352  26772.894079    26773.383785   
                     98              342  26784.748589    26785.216325   
                     99              251  26793.602534    26793.833890   

                           scenario_done_ts  init_time  scenario_time  \
dt   decision_repeat seed                                               
0.02 5               0         26007.531759   0.468928   2.140005e-07   
                     1         26015.015511   0.309328   2.870001e-07   
                     2         26020.946455   0.360457   2.010020e-07   
                     3         26026.851899   0.281732   2.460001e-07   
                     4         26031.908182   0.261337   2.029992e-07   
...                                     ...        ...            ...   
0.04 20              95        26751.508414   0.361998   2.829984e-07   
                     96        26761.216448   0.293364   2.289999e-07   
                     97        26773.383785   0.489705   2.960005e-07   
                     98        26785.216325   0.467736   2.279994e-07   
                     99        26793.833891   0.231357   2.279994e-07   

                           total_time  n_crashed_vehicles  \
dt   deci

In [9]:
# calculate driving score
# ! Problem changing the values here in the analysis can change % error
def calculate_driving_score(df):
    df["driving_score"] = (
        df["route_completion"]
        * df["in_road_percentage"]
        * (0.65) ** df["n_sidewalk_crashes"]
        * (0.60) ** df["n_crashed_vehicles"]
    )
    return df

In [69]:
df["def.map_seq"].reset_index()

,dt,decision_repeat,seed,def.map_seq
0,0.02,5,0,"[{'id': 'I', 'pre_block_socket_index': None}, ..."
1,0.02,5,1,"[{'id': 'I', 'pre_block_socket_index': None}, ..."
2,0.02,5,2,"[{'id': 'I', 'pre_block_socket_index': None}, ..."
3,0.02,5,3,"[{'id': 'I', 'pre_block_socket_index': None}, ..."
4,0.02,5,4,"[{'id': 'I', 'pre_block_socket_index': None}, ..."
...,...,...,...,...
1195,0.04,20,95,"[{'id': 'I', 'pre_block_socket_index': None}, ..."
1196,0.04,20,96,"[{'id': 'I', 'pre_block_socket_index': None}, ..."
1197,0.04,20,97,"[{'id': 'I', 'pre_block_socket_index': None}, ..."
1198,0.04,20,98,"[{'id': 'I', 'pre_block_socket_index': None}, ..."


In [68]:
map_df = pd.json_normalize(df["def.map_seq"])
map_df

,0,1,2,3,4,5
0,"{'id': 'I', 'pre_block_socket_index': None}","{'radius': 56.134971618652344, 'dir': 1, 'leng...","{'radius': 35.112525939941406, 'dir': 0, 'leng...","{'radius': 45.43406295776367, 'dir': 1, 'lengt...","{'change_lane_num': 0, 'decrease_increase': 1,...","{'radius': 40.848209381103516, 'dir': 0, 'leng..."
1,"{'id': 'I', 'pre_block_socket_index': None}","{'change_lane_num': 0, 't_type': 1, 'decrease_...","{'change_lane_num': 0, 'decrease_increase': 1,...","{'change_lane_num': 0, 'decrease_increase': 1,...","{'change_lane_num': 0, 't_type': 0, 'decrease_...","{'length': 65.34033203125, 'id': 'S', 'pre_blo..."
2,"{'id': 'I', 'pre_block_socket_index': None}","{'change_lane_num': 0, 't_type': 0, 'decrease_...","{'radius': 50.500247955322266, 'dir': 1, 'leng...","{'change_lane_num': 0, 'decrease_increase': 1,...","{'radius': 58.98457336425781, 'dir': 1, 'lengt...","{'length': 26.290544509887695, 'id': 'r', 'pre..."
3,"{'id': 'I', 'pre_block_socket_index': None}","{'change_lane_num': 0, 'decrease_increase': 1,...","{'change_lane_num': 0, 'decrease_increase': 0,...","{'radius': 35.4585075378418, 'dir': 0, 'length...","{'radius': 33.10982894897461, 'dir': 0, 'lengt...","{'change_lane_num': 0, 't_type': 1, 'decrease_..."
4,"{'id': 'I', 'pre_block_socket_index': None}","{'change_lane_num': 0, 't_type': 2, 'decrease_...","{'radius': 54.780181884765625, 'dir': 1, 'leng...","{'change_lane_num': 0, 't_type': 2, 'decrease_...","{'change_lane_num': 0, 't_type': 0, 'decrease_...","{'length': 28.557395935058594, 'id': 'R', 'pre..."
...,...,...,...,...,...,...
1195,"{'id': 'I', 'pre_block_socket_index': None}","{'inner_radius': 16.645484924316406, 'exit_rad...","{'radius': 50.452247619628906, 'dir': 1, 'leng...","{'length': 46.848388671875, 'id': 'S', 'pre_bl...","{'radius': 58.217960357666016, 'dir': 1, 'leng...","{'radius': 38.35463333129883, 'dir': 0, 'lengt..."
1196,"{'id': 'I', 'pre_block_socket_index': None}","{'change_lane_num': 0, 't_type': 0, 'decrease_...","{'radius': 53.45768356323242, 'dir': 1, 'lengt...","{'change_lane_num': 0, 't_type': 1, 'decrease_...","{'change_lane_num': 0, 't_type': 0, 'decrease_...","{'length': 79.40625, 'id': 'S', 'pre_block_soc..."
1197,"{'id': 'I', 'pre_block_socket_index': None}","{'inner_radius': 20.39239501953125, 'exit_radi...","{'change_lane_num': 0, 'decrease_increase': 1,...","{'length': 26.95365333557129, 'id': 'r', 'pre_...","{'inner_radius': 40.184349060058594, 'exit_rad...","{'change_lane_num': 0, 'decrease_increase': 0,..."
1198,"{'id': 'I', 'pre_block_socket_index': None}","{'change_lane_num': 0, 't_type': 0, 'decrease_...","{'length': 33.736576080322266, 'id': 'R', 'pre...","{'change_lane_num': 0, 'decrease_increase': 1,...","{'inner_radius': 43.49483871459961, 'exit_radi...","{'inner_radius': 35.790435791015625, 'exit_rad..."


In [70]:
exploded_df = pd.concat(
    [pd.json_normalize(map_df[col]).add_prefix(f"{col}_") for col in map_df], axis=1
)
exploded_df

,0_id,0_pre_block_socket_index,1_radius,1_dir,1_length,1_angle,1_id,1_pre_block_socket_index,1_change_lane_num,1_t_type,...,5_dir,5_length,5_angle,5_id,5_pre_block_socket_index,5_change_lane_num,5_t_type,5_decrease_increase,5_inner_radius,5_exit_radius
0,I,None,56.134972,1.0,75.582825,125.061348,C,0I-socket0,NaN,NaN,...,0.0,58.112240,85.752541,C,4X-socket0,NaN,NaN,NaN,NaN,NaN
1,I,None,10.000000,NaN,NaN,NaN,T,0I-socket0,0.0,1.0,...,NaN,65.340332,NaN,S,4T-socket1,NaN,NaN,NaN,NaN,NaN
2,I,None,10.000000,NaN,NaN,NaN,T,0I-socket0,0.0,0.0,...,NaN,26.290545,NaN,r,4C-socket0,NaN,NaN,NaN,NaN,NaN
3,I,None,10.000000,NaN,NaN,NaN,X,0I-socket0,0.0,NaN,...,NaN,NaN,NaN,T,4C-socket0,0.0,1.0,1.0,NaN,NaN
4,I,None,10.000000,NaN,NaN,NaN,T,0I-socket0,0.0,2.0,...,NaN,28.557396,NaN,R,4T-socket2,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,I,None,NaN,NaN,NaN,60.000000,O,0I-socket0,NaN,NaN,...,0.0,55.262436,79.340485,C,4C-socket0,NaN,NaN,NaN,NaN,NaN
1196,I,None,10.000000,NaN,NaN,NaN,T,0I-socket0,0.0,0.0,...,NaN,79.406250,NaN,S,4T-socket2,NaN,NaN,NaN,NaN,NaN
1197,I,None,NaN,NaN,NaN,60.000000,O,0I-socket0,NaN,NaN,...,NaN,NaN,NaN,X,4O-socket1,0.0,NaN,0.0,NaN,NaN
1198,I,None,10.000000,NaN,NaN,NaN,T,0I-socket0,0.0,0.0,...,NaN,NaN,60.000000,O,4O-socket1,NaN,NaN,NaN,35.790436,11.930146


In [77]:
dummies_df = pd.get_dummies(exploded_df)
dummies_df
new_df = df["def.max_steps"].reset_index().join(dummies_df)

,dt,decision_repeat,seed,def.max_steps,1_radius,1_dir,1_length,1_angle,1_change_lane_num,1_t_type,...,5_pre_block_socket_index_4O-socket2,5_pre_block_socket_index_4R-socket0,5_pre_block_socket_index_4S-socket0,5_pre_block_socket_index_4T-socket0,5_pre_block_socket_index_4T-socket1,5_pre_block_socket_index_4T-socket2,5_pre_block_socket_index_4X-socket0,5_pre_block_socket_index_4X-socket1,5_pre_block_socket_index_4X-socket2,5_pre_block_socket_index_4r-socket0
0,0.02,5,0,3364,56.134972,1.0,75.582825,125.061348,NaN,NaN,...,False,False,False,False,False,False,True,False,False,False
1,0.02,5,1,1892,10.000000,NaN,NaN,NaN,0.0,1.0,...,False,False,False,False,True,False,False,False,False,False
2,0.02,5,2,3574,10.000000,NaN,NaN,NaN,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,0.02,5,3,2092,10.000000,NaN,NaN,NaN,0.0,NaN,...,False,False,False,False,False,False,False,False,False,False
4,0.02,5,4,2815,10.000000,NaN,NaN,NaN,0.0,2.0,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,0.04,20,95,423,NaN,NaN,NaN,60.000000,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
1196,0.04,20,96,330,10.000000,NaN,NaN,NaN,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False
1197,0.04,20,97,352,NaN,NaN,NaN,60.000000,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
1198,0.04,20,98,342,10.000000,NaN,NaN,NaN,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False


In [87]:
bv_df = pd.json_normalize(df["def.bv_data"])
bv_df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,"{'length': 4.515, 'width': 1.852, 'height': 1....","{'length': 4.6, 'width': 1.85, 'height': 1.37,...","{'length': 5.74, 'width': 2.3, 'height': 2.8, ...","{'length': 4.87, 'width': 2.046, 'height': 1.8...","{'length': 5.74, 'width': 2.3, 'height': 2.8, ...","{'length': 4.6, 'width': 1.85, 'height': 1.37,...","{'length': 4.3, 'width': 1.7, 'height': 1.7, '...","{'length': 4.3, 'width': 1.7, 'height': 1.7, '...","{'length': 4.3, 'width': 1.7, 'height': 1.7, '...","{'length': 4.6, 'width': 1.85, 'height': 1.37,...",...,"{'length': 4.6, 'width': 1.85, 'height': 1.37,...","{'length': 4.6, 'width': 1.85, 'height': 1.37,...","{'length': 4.6, 'width': 1.85, 'height': 1.37,...","{'length': 4.87, 'width': 2.046, 'height': 1.8...","{'length': 4.87, 'width': 2.046, 'height': 1.8...","{'length': 4.3, 'width': 1.7, 'height': 1.7, '...","{'length': 4.87, 'width': 2.046, 'height': 1.8...",None,None,None
1,"{'length': 4.515, 'width': 1.852, 'height': 1....","{'length': 5.74, 'width': 2.3, 'height': 2.8, ...","{'length': 4.6, 'width': 1.85, 'height': 1.37,...","{'length': 4.87, 'width': 2.046, 'height': 1.8...","{'length': 4.3, 'width': 1.7, 'height': 1.7, '...","{'length': 4.6, 'width': 1.85, 'height': 1.37,...","{'length': 4.87, 'width': 2.046, 'height': 1.8...","{'length': 4.3, 'width': 1.7, 'height': 1.7, '...",None,None,...,None,None,None,None,None,None,None,None,None,None
2,"{'length': 4.515, 'width': 1.852, 'height': 1....","{'length': 4.6, 'width': 1.85, 'height': 1.37,...","{'length': 5.74, 'width': 2.3, 'height': 2.8, ...","{'length': 4.87, 'width': 2.046, 'height': 1.8...","{'length': 4.87, 'width': 2.046, 'height': 1.8...","{'length': 4.3, 'width': 1.7, 'height': 1.7, '...","{'length': 4.6, 'width': 1.85, 'height': 1.37,...","{'length': 4.87, 'width': 2.046, 'height': 1.8...","{'length': 4.3, 'width': 1.7, 'height': 1.7, '...","{'length': 4.6, 'width': 1.85, 'height': 1.37,...",...,"{'length': 4.87, 'width': 2.046, 'height': 1.8...","{'length': 5.74, 'width': 2.3, 'height': 2.8, ...","{'length': 4.3, 'width': 1.7, 'height': 1.7, '...","{'length': 5.74, 'width': 2.3, 'height': 2.8, ...","{'length': 4.87, 'width': 2.046, 'height': 1.8...","{'length': 5.74, 'width': 2.3, 'height': 2.8, ...","{'length': 4.87, 'width': 2.046, 'height': 1.8...",None,None,None
3,"{'length': 4.515, 'width': 1.852, 'height': 1....","{'length': 4.3, 'width': 1.7, 'height': 1.7, '...","{'length': 4.6, 'width': 1.85, 'height': 1.37,...","{'length': 4.87, 'width': 2.046, 'height': 1.8...","{'length': 4.87, 'width': 2.046, 'height': 1.8...","{'length': 4.87, 'width': 2.046, 'height': 1.8...","{'length': 4.6, 'width': 1.85, 'height': 1.37,...","{'length': 4.87, 'width': 2.046, 'height': 1.8...","{'length': 4.3, 'width': 1.7, 'height': 1.7, '...","{'length': 4.87, 'width': 2.046, 'height': 1.8...",...,None,None,None,None,None,None,None,None,None,None
4,"{'length': 4.515, 'width': 1.852, 'height': 1....","{'length': 4.87, 'width': 2.046, 'height': 1.8...","{'length': 4.3, 'width': 1.7, 'height': 1.7, '...","{'length': 4.6, 'width': 1.85, 'height': 1.37,...","{'length': 4.6, 'width': 1.85, 'height': 1.37,...","{'length': 4.87, 'width': 2.046, 'height': 1.8...","{'length': 4.6, 'width': 1.85, 'height': 1.37,...","{'length': 4.3, 'width': 1.7, 'height': 1.7, '...","{'length': 4.6, 'width': 1.85, 'height': 1.37,...","{'length': 5.74, 'width': 2.3, 'height': 2.8, ...",...,"{'length': 4.87, 'width': 2.046, 'height': 1.8...",None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,"{'length': 4.515, 'width': 1.852, 'height': 1....","{'length': 5.74, 'width': 2.3, 'height': 2.8, ...","{'length': 4.87, 'width': 2.046, 'height': 1.8...","{'length': 4.6, 'width': 1.85, 'height': 1.37,...","{'length': 4.3, 'width': 1.7, 'height': 1.7, '...","{'length': 5.74, 'width': 2.3, 'height': 2.8, ...","{'length': 4.6, 'width': 1.85, 'height': 1.37,..

In [91]:
exploded_df = pd.concat(
    [pd.json_normalize(bv_df[col]).add_prefix(f"{col}_") for col in bv_df], axis=1
)
exploded_df

,0_length,0_width,0_height,0_spawn_road,0_destination,1_length,1_width,1_height,1_spawn_road,1_destination,...,19_length,19_width,19_height,19_spawn_road,19_destination,20_length,20_width,20_height,20_spawn_road,20_destination
0,4.515,1.852,1.19,"[>, >>]","[5C0_0_, 5C0_1_]",4.60,1.850,1.37,"[1C0_0_, 1C0_1_]","[5C0_0_, 5C0_1_]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.515,1.852,1.19,"[>, >>]","[4T1_1_, 5S0_0_]",5.74,2.300,2.80,"[-1T2_1_, -1T2_0_]","[->>>, ->>]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.515,1.852,1.19,"[>, >>]","[5r0_1_, 5r0_2_]",4.60,1.850,1.37,"[-1T2_1_, -1T2_0_]","[->>>, ->>]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.515,1.852,1.19,"[>, >>]","[5T2_0_, 5T2_1_]",4.30,1.700,1.70,"[-1X0_1_, -1X0_0_]","[->>>, ->>]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.515,1.852,1.19,"[>, >>]","[5R0_0_, 5R0_1_]",4.87,2.046,1.85,"[-1T0_1_, -1T0_0_]","[->>>, ->>]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,4.515,1.852,1.19,"[>, >>]","[5C0_0_, 5C0_1_]",5.74,2.300,2.80,"[-1O1_3_, -1O1_2_]","[->>>, ->>]",...,4.3,1.7,1.7,"[5C0_0_, 5C0_1_]","[5C0_0_, 5C0_1_]",NaN,NaN,NaN,NaN,NaN
1196,4.515,1.852,1.19,"[>, >>]","[4T2_1_, 5S0_0_]",4.30,1.700,1.70,"[-1T2_1_, -1T2_0_]","[->>>, ->>]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197,4.515,1.852,1.19,"[>, >>]","[5X2_0_, 5X2_1_]",4.30,1.700,1.70,"[-1O2_3_, -1O2_2_]","[->>>, ->>]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1198,4.515,1.852,1.19,"[>, >>]","[5O0_2_, 5O0_3_]",5.74,2.300,2.80,"[-1T1_1_, -1T1_0_]","[->>>, ->>]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
dummies_df = pd.get_dummies(exploded_df)
dummies_df

TypeError: unhashable type: 'list'